In [12]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_KEY')
os.environ["NEO4J_URI"] = os.getenv('NEO4J_URL')
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = os.getenv('NEO4J_PW')
os.environ["NEO4J_DATABSE"] = "neo4j"

In [13]:
from langchain_openai import ChatOpenAI
from langchain_community.graphs import Neo4jGraph

# url = os.getenv('NEO4J_URL')
# username = "neo4j"
# password = os.getenv('NEO4J_PW')
# database = "neo4j"

llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
graph = Neo4jGraph()

In [14]:
from langchain_core.prompts import ChatPromptTemplate
from retrieve import Entities

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following"
            "input: {question}",
        ),
    ]
)

entity_chain = prompt | llm.with_structured_output(Entities)

In [15]:
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings

vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

graph.query("CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")


[]

In [17]:
from retrieve import structured_retriever

print(structured_retriever("Tell me about Austin"))